# How to save data from FPL API into MongoDB

In [1]:
import config
import requests
import time
from pprint import pprint
from pymongo import MongoClient
from tqdm.auto import tqdm

In [2]:
BASE_URL = 'https://fantasy.premierleague.com/api/'
API_DATA = requests.get(BASE_URL+'bootstrap-static/').json()

In [17]:
# Bootstrap-static data
pprint(API_DATA, depth=1)

{'element_stats': [...],
 'element_types': [...],
 'elements': [...],
 'events': [...],
 'fixtures': [...],
 'game_settings': {...},
 'phases': [...],
 'teams': [...],
 'total_players': 11431471}


In [18]:
# connect to MongoDB
client = MongoClient(f'mongodb+srv://{config.username}:{config.password}'
                     f'@{config.database}/?retryWrites=true&w=majority')
db = client.get_database('raw_2022-23')

In [19]:
def get_bootstrap_static():
    '''Get all data from the bootstrap-static endpoint and update MongoDB
    
    Bootstrap-static data includes the following:
      - elements (players)
      - element_types (positions)
      - events (gameweeks)
      - teams
    '''

    # delete unnecessary keys
    for key in ['element_stats', 'game_settings', 'total_players', 'phases']:
        del API_DATA[key]

    # iterate over each endpoint
    for endpoint in API_DATA.keys():
        print(f'Updating {endpoint.upper()} data')
        data = API_DATA[endpoint].copy()

        # get number of docs, cols in collection
        n_docs = db.get_collection(endpoint).count_documents({})
        n_cols = len(db[endpoint].find_one({})) if n_docs else 0
        print(f'{endpoint} shape: {n_docs} docs X {n_cols} cols')

        # update docs in each collection
        for row in tqdm(data):
            row['_id'] = row.pop('id')  # rename id field
            filter_query = {'_id': row['_id']}
            update_data = {'$set': row}
            db[endpoint].update_one(filter_query, update_data, upsert=True)

        # print number of new docs, cols
        print(f'Added {db[endpoint].count_documents({}) - n_docs} new docs')
        print(f'Added {len(db[endpoint].find_one({})) - n_cols} new cols\n')

    return True

In [20]:
get_bootstrap_static()

Updating events data
events shape: 38 docs X 23 cols


  0%|          | 0/38 [00:00<?, ?it/s]

Added 0 new docs
Added 0 new cols

Updating teams data
teams shape: 20 docs X 21 cols


  0%|          | 0/20 [00:00<?, ?it/s]

Added 0 new docs
Added 0 new cols

Updating elements data
elements shape: 764 docs X 88 cols


...

  0%|          | 0/4 [00:00<?, ?it/s]

Added 0 new docs
Added 0 new cols

Updating fixtures data
fixtures shape: 380 docs X 10 cols


  0%|          | 0/380 [00:00<?, ?it/s]

Added 0 new docs
Added 0 new cols



True